In [39]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from scipy.spatial.distance import cosine
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
import scipy.stats as stats

In [40]:
performance = pd.read_csv('GravAttnLEHD-spatial-transfer-radiation-euclidean-GAT-resnet-GAN.csv')
performance = performance.iloc[::5,:]
performance.columns = ['source+target']+[i.split(' KL')[0] for i in performance['source-target'].values]
performance = performance.reset_index()
del performance['index']

In [41]:
for i in range(1,13):
    performance[performance.columns[i]] = performance[performance.columns[i]][i-1] / performance[performance.columns[i]]

In [42]:
performance.iloc[:,1:].values.flatten()

array([1.        , 1.00195695, 0.98474341, 0.99538462, 0.99911817,
       0.98677249, 1.00093371, 0.98381601, 0.95285858, 0.98743719,
       0.84948352, 1.00079177, 0.95325203, 1.        , 0.99719101,
       1.        , 0.97756687, 0.98416887, 0.97543221, 1.00697472,
       0.99789916, 1.00255102, 0.98968481, 0.98827209, 0.96502058,
       1.00392157, 1.        , 1.        , 0.98521739, 0.98807947,
       0.97989031, 1.00260417, 0.98752599, 1.        , 0.96859226,
       0.99605989, 0.93240557, 0.97709924, 0.98201936, 1.        ,
       0.93869097, 0.99070385, 0.95799821, 1.00260417, 0.99789916,
       0.99242424, 1.01767826, 0.95612708, 0.97505198, 1.00589391,
       0.99859353, 0.99845679, 1.        , 0.98546896, 0.99167438,
       0.99654875, 0.96839959, 0.99493671, 0.92303581, 1.00158479,
       0.91423002, 0.94990724, 0.95687332, 0.98929664, 0.89353312,
       1.        , 0.94200351, 0.98130841, 0.96446701, 0.96560197,
       1.01767826, 0.91594203, 0.98945148, 1.00392157, 0.98885

In [43]:
performance

,source+target,New York City,Los Angeles,Chicago,Houston,Boston,Phoenix,Philadelphia,San Antonio,San Diego,Dallas,San Jose,Austin
0,New York City KL,1.000000,1.001957,0.984743,0.995385,0.999118,0.986772,1.000934,0.983816,0.952859,0.987437,0.849484,1.000792
1,Los Angeles KL,0.953252,1.000000,0.997191,1.000000,0.977567,0.984169,0.975432,1.006975,0.997899,1.002551,0.989685,0.988272
2,Chicago KL,0.965021,1.003922,1.000000,1.000000,0.985217,0.988079,0.979890,1.002604,0.987526,1.000000,0.968592,0.996060
3,Houston KL,0.932406,0.977099,0.982019,1.000000,0.938691,0.990704,0.957998,1.002604,0.997899,0.992424,1.017678,0.956127
4,Boston KL,0.975052,1.005894,0.998594,0.998457,1.000000,0.985469,0.991674,0.996549,0.968400,0.994937,0.923036,1.001585
5,Phoenix KL,0.914230,0.949907,0.956873,0.989297,0.893533,1.000000,0.942004,0.981308,0.964467,0.965602,1.017678,0.915942
6,Philadelphia KL,0.989451,1.003922,0.988858,0.996918,0.999118,0.985469,1.000000,0.988024,0.951904,0.991173,0.860917,1.000792
7,San Antonio KL,0.936128,0.975238,0.980663,1.000000,0.938691,0.993342,0.960573,1.000000,0.995807,0.991173,1.018879,0.955404
8,San Diego KL,0.953252,0.992248,0.995792,1.000000,0.969204,0.990704,0.972777,1.007853,1.000000,1.001274,0.997689,0.982129
9,Dallas KL,0.947475,0.990329,0.990237,1.000000,0.964255,0.986772,0.969259,1.007853,1.002110,1.000000,1.003486,0.976062


In [44]:

splitByNodes = True
externalities = True
# if modelmode='g', conventional singly constrained gravity model, other parameters will be ignored
modelmode = 'p'
# gravi con, modelmode = 'p', attention = True, directEmebdding = False
# GCN + MLP, modelmode = 'p', attention = False, VNNattraction = Truepath = 'LEHD/'
seed = 0
cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]
t_cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]



node_similarity = []
A_similarity = []
pop_similarity = []
job_similarity = []
bet_similarity1 = []
bet_similarity2 = []
bet_similarity3 = []
for city,state, counties in cities:
    splitSeed = 0
    seed = 0

    mainepochs = 500

    lr = [1e-3]
        

    #some basic stats
    print('loading datasets')
    A_train = torch.load('training/'+city+'_A_train.pt')
    OD_train = torch.load('training/'+city+'_OD_train.pt')
    between_fts_train = torch.load('training/'+city+'_between_fts_train.pt')
    nodefts_train = torch.load('training/'+city+'_nodefts_train.pt')
    pop_train = OD_train.sum(axis = 1)
    job_train = OD_train.sum(axis = 0)

#     nodefts_train_t = scaler.transform(nodefts_train)
    node_base_cosine_similarity = cosine_similarity(nodefts_train,nodefts_train).mean()

    for target_city,target_state, target_counties in t_cities:

        print('loading target LEHD:',target_city)
        target_A = torch.load('training/'+target_city+'_A_train.pt')
        target_OD = torch.load('training/'+target_city+'_OD_train.pt')
        target_between_fts = torch.load('training/'+target_city+'_between_fts_train.pt')
        target_nodefts = torch.load('training/'+target_city+'_nodefts_train.pt')
        target_pop = target_OD.sum(axis = 1)
        target_job = target_OD.sum(axis = 0)
        # standardization
        

#         target_nodefts_t = scaler.transform(target_nodefts)

        node_similarity += [cosine_similarity(nodefts_train,target_nodefts).mean()/node_base_cosine_similarity]

        A_similarity += [ks_2samp(A_train.flatten(),target_A.flatten())[0]]
        pop_similarity += [ks_2samp(pop_train,target_pop)[0]]
        job_similarity += [ks_2samp(job_train,target_job)[0]]
        bet_similarity1 += [ks_2samp(between_fts_train[:,0],target_between_fts[:,0])[0]]
        bet_similarity2 += [ks_2samp(between_fts_train[:,1],target_between_fts[:,1])[0]]
        bet_similarity3 += [ks_2samp(between_fts_train[:,2],target_between_fts[:,2])[0]]



loading datasets
loading target LEHD: New York City
loading target LEHD: Los Angeles
loading target LEHD: Chicago
loading target LEHD: Houston
loading target LEHD: Boston
loading target LEHD: Phoenix
loading target LEHD: Philadelphia
loading target LEHD: San Antonio
loading target LEHD: San Diego
loading target LEHD: Dallas
loading target LEHD: San Jose
loading target LEHD: Austin
loading datasets
loading target LEHD: New York City
loading target LEHD: Los Angeles
loading target LEHD: Chicago
loading target LEHD: Houston
loading target LEHD: Boston
loading target LEHD: Phoenix
loading target LEHD: Philadelphia
loading target LEHD: San Antonio
loading target LEHD: San Diego
loading target LEHD: Dallas
loading target LEHD: San Jose
loading target LEHD: Austin
loading datasets
loading target LEHD: New York City
loading target LEHD: Los Angeles
loading target LEHD: Chicago
loading target LEHD: Houston
loading target LEHD: Boston
loading target LEHD: Phoenix
loading target LEHD: Philadelphi

In [45]:
city = 'New York City'
fts = pd.read_csv('LEHD/%sfts.csv'%city)
columns = fts.columns[1:]

In [46]:
columns

Index(['transportation', 'restaurant', 'health', 'education', 'retail', 'C000',
       'CA01', 'CA02', 'CA03', 'CE01', 'CE02', 'CE03', 'CNS01', 'CNS02',
       'CNS03', 'CNS04', 'CNS05', 'CNS06', 'CNS07', 'CNS08', 'CNS09', 'CNS10',
       'CNS11', 'CNS12', 'CNS13', 'CNS14', 'CNS15', 'CNS16', 'CNS17', 'CNS18',
       'CNS19', 'CNS20', 'CR01', 'CR02', 'CR03', 'CR04', 'CR05', 'CR07',
       'CT01', 'CT02', 'CD01', 'CD02', 'CD03', 'CD04', 'CS01', 'CS02'],
      dtype='object')

In [47]:

splitByNodes = True
externalities = True
# if modelmode='g', conventional singly constrained gravity model, other parameters will be ignored
modelmode = 'p'
# gravi con, modelmode = 'p', attention = True, directEmebdding = False
# GCN + MLP, modelmode = 'p', attention = False, VNNattraction = Truepath = 'LEHD/'
seed = 0
cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]
t_cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

def jaccard_similarity(set1, set2):
    # intersection of two sets
    intersection = len(set1.intersection(set2))
    # Unions of two sets
    union = len(set1.union(set2))
     
    return intersection / union




for i in range(46):
    node_similarity = []
    for city,state, counties in cities:



        #some basic stats
#         print('loading datasets')

        nodefts_train = torch.load('training/'+city+'_nodefts_train.pt')


    #     nodefts_train_t = scaler.transform(nodefts_train)

        for target_city,target_state, target_counties in t_cities:


            target_nodefts = torch.load('training/'+target_city+'_nodefts_train.pt')
            
            node_similarity += [-ks_2samp(nodefts_train[:,i],target_nodefts[:,i])[0]]
    print(columns[i],',',round(stats.pearsonr(np.array(node_similarity),
                     performance.iloc[:,1:].values.flatten())[0],3),',',round(stats.pearsonr(np.array(node_similarity),
                     performance.iloc[:,1:].values.flatten())[1],3))
        

transportation , 0.084 , 0.316
restaurant , 0.13 , 0.12
health , 0.095 , 0.257
education , 0.125 , 0.134
retail , 0.392 , 0.0
C000 , 0.22 , 0.008
CA01 , 0.198 , 0.017
CA02 , 0.175 , 0.036
CA03 , 0.28 , 0.001
CE01 , 0.285 , 0.001
CE02 , 0.193 , 0.021
CE03 , 0.13 , 0.121
CNS01 , 0.323 , 0.0
CNS02 , -0.088 , 0.294
CNS03 , 0.084 , 0.317
CNS04 , 0.192 , 0.021
CNS05 , 0.269 , 0.001
CNS06 , 0.021 , 0.801
CNS07 , 0.237 , 0.004
CNS08 , 0.093 , 0.268
CNS09 , 0.22 , 0.008
CNS10 , 0.242 , 0.004
CNS11 , 0.07 , 0.405
CNS12 , 0.319 , 0.0
CNS13 , 0.14 , 0.094
CNS14 , 0.291 , 0.0
CNS15 , 0.029 , 0.733
CNS16 , 0.027 , 0.746
CNS17 , 0.259 , 0.002
CNS18 , 0.336 , 0.0
CNS19 , 0.2 , 0.017
CNS20 , 0.347 , 0.0
CR01 , 0.228 , 0.006
CR02 , 0.353 , 0.0
CR03 , 0.341 , 0.0
CR04 , 0.155 , 0.064
CR05 , 0.189 , 0.023
CR07 , 0.178 , 0.033
CT01 , 0.099 , 0.238
CT02 , 0.206 , 0.013
CD01 , 0.213 , 0.01
CD02 , 0.097 , 0.249
CD03 , 0.207 , 0.013
CD04 , 0.123 , 0.143
CS01 , 0.174 , 0.037
CS02 , 0.213 , 0.011


In [48]:
stats.pearsonr(np.array(node_similarity),
                     performance.iloc[:,1:].values.flatten())

PearsonRResult(statistic=0.21253388811349622, pvalue=0.01054377231416265)

In [49]:
def read_area_from_ct(state,nodes):
    ct_map = gpd.read_file('LEHD/'+state+'.zip')
    ct_map['GEOID'] = ct_map['GEOID'].astype(int)
    ct_map = ct_map.set_index('GEOID')
    ct_map = ct_map.loc[list(nodes.keys())]

    return ct_map.to_crs('ESRI:102008').geometry.area.values

In [50]:

splitByNodes = True
externalities = True
# if modelmode='g', conventional singly constrained gravity model, other parameters will be ignored
modelmode = 'p'
# gravi con, modelmode = 'p', attention = True, directEmebdding = False
# GCN + MLP, modelmode = 'p', attention = False, VNNattraction = Truepath = 'LEHD/'
seed = 0
cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]
t_cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

def jaccard_similarity(set1, set2):
    # intersection of two sets
    intersection = len(set1.intersection(set2))
    # Unions of two sets
    union = len(set1.union(set2))
     
    return intersection / union




pop_similarity = []
job_similarity = []
area_similarity = []
for city,state, counties in cities:



    #some basic stats
#         print('loading datasets')

    area_train = torch.load('training/'+city+'_area_train.pt')
    OD_train = torch.load('training/'+city+'_OD_train.pt')
    pop_train = OD_train.sum(axis = 1)
    job_train = OD_train.sum(axis = 0)

#     nodefts_train_t = scaler.transform(nodefts_train)

    for target_city,target_state, target_counties in t_cities:


        area_target = torch.load('training/'+target_city+'_area_train.pt')
        OD_target = torch.load('training/'+target_city+'_OD_train.pt')
        pop_target = OD_target.sum(axis = 1)
        job_target = OD_target.sum(axis = 0)

        area_similarity += [-ks_2samp(area_train,area_target)[0]]
        pop_similarity += [-ks_2samp(pop_train/area_train,pop_target/area_target)[0]]
        job_similarity += [-ks_2samp(job_train/area_train,job_target/area_target)[0]]
print('area',stats.pearsonr(np.array(area_similarity),
                 performance.iloc[:,1:].values.flatten()))
print('pop',stats.pearsonr(np.array(pop_similarity),
                 performance.iloc[:,1:].values.flatten()))
print('job',stats.pearsonr(np.array(job_similarity),
                 performance.iloc[:,1:].values.flatten()))
        

area PearsonRResult(statistic=0.3772829509717233, pvalue=3.1409133061458397e-06)
pop PearsonRResult(statistic=0.31951658393854987, pvalue=9.47669289133253e-05)
job PearsonRResult(statistic=0.3388971591440351, pvalue=3.253479238694084e-05)
